# Get data from cemdata18, generate reactions and export them to chess/phreeqc format

In [1]:
import thermohubclient as client
import thermofun as fun
import chemicalfun as cf
import json
from common.filters import substances_containing_elements
from common.export import to_chess, to_phreeqc

In [2]:
dbc = client.DatabaseClient()

In [3]:
# what are the available datasets in ThermoHub 
dbc.availableThermoDataSets()

['"mines19"',
 '"heracles"',
 '"slop98-organic"',
 '"slop16"',
 '"aq17"',
 '"psinagra-12-07"',
 '"slop98-inorganic"',
 '"mines16"',
 '"cemdata18"']

In [4]:
# get cemdata dataset
dbc.saveDatabase("cemdata18")
# cemdata18 dataset is downloaded from ThemroHub server and saved as cemdata18-thermofun.json

In [5]:
# what substances are avaialble containing Si and Fe elements
substances_containing_elements('cemdata18-thermofun.json', ['Fe', ''])

symbol               formula                                  class               
Fe2(OH)2+4           Fe|3|2(OH)2+4                            SC_AQSOLUTE         
Fe3(OH)4+5           Fe|3|3(OH)4+5                            SC_AQSOLUTE         
FeHSiO3+2            Fe|3|HSiO3+2                             SC_AQSOLUTE         
FeCl3@               Fe|3|Cl3@                                SC_AQSOLUTE         
Fe(SO4)@             Fe(SO4)@                                 SC_AQSOLUTE         
Fe(SO4)+             Fe|3|(SO4)+                              SC_AQSOLUTE         
Fe(SO4)2-            Fe|3|(SO4)2-                             SC_AQSOLUTE         
Fe(CO3)@             FeCO3@                                   SC_AQSOLUTE         
Fe(HCO3)+            FeHCO3+                                  SC_AQSOLUTE         
FeCl2+               Fe|3|Cl2+                                SC_AQSOLUTE         
Fe(HSO4)+2           Fe|3|HSO4+2                              SC_AQSOLUTE         
Fe(H

In [6]:
# create a ThermoFun database from which we can select the substances to generate reactions
fun_cemdata18 = fun.Database('cemdata18-thermofun.json')

# extract preferred substances from database 
CAFS = fun_cemdata18.getSubstance("C3AFS0.84H4.32")
CFS = fun_cemdata18.getSubstance("C3FS0.84H4.32")
CFS2 = fun_cemdata18.getSubstance("C3FS1.34H3.32")
Fe_hemi = fun_cemdata18.getSubstance("Fe-hemicarbonate")
hem = fun_cemdata18.getSubstance("Hem")

substances = [CAFS, CFS, CFS2, Fe_hemi, hem]
aqueous_master_species = ['H2O@', 'Al+3', 'Fe+3', 'SiO2@', 'Ca+2', 'H+', 'HCO3-']

In [7]:
# using chemicalfun library to generate chemical reactions based on list of master species
substances_formulas = aqueous_master_species+[s.formula() for s in substances]
substances_symbols = aqueous_master_species+[s.symbol() for s in substances]

chemicalReactions = cf.ChemicalReactions(substances_formulas,substances_symbols)

reactions = chemicalReactions.generateReactions(formation=True) # returns the reactions list as a list of tuples ('substance', coefficient)

# list of dictionaries, with reaction substances as keys and the reaction coefficients as values
reactions_dic = [{el[0]: el[1] for el in r} for r in reactions] 

# strings of reactions that can be used in ThermoFun to calculate the logK at different T and P
reactions_list = chemicalReactions.stringReactions()

for r in reactions_list:
    print(r)
#reactions_dic

3Ca+2 + 0.84SiO2@ + Fe+3 + Al+3 + 10.32H2O@ = C3AFS0.84H4.32 + 12H+
3Ca+2 + 0.84SiO2@ + 2Fe+3 + 10.32H2O@ = C3FS0.84H4.32 + 12H+
3Ca+2 + 1.34SiO2@ + 2Fe+3 + 9.32H2O@ = C3FS1.34H3.32 + 12H+
0.5HCO3- + 4Ca+2 + 2Fe+3 + 16.5H2O@ = Fe-hemicarbonate + 13.5H+
2Fe+3 + 3H2O@ = Hem + 6H+


In [8]:
# calculating reaction properties for given T list
T_list = [0,25,60,100,150,200,250, 300]
engine = fun.ThermoEngine(fun_cemdata18) # thermofun engine is used to calculate the properties of reactions
# collecting results from the loop
reactions_logKs =[ [engine.thermoPropertiesReaction(t+273.1501, 0, r).log_equilibrium_constant.val for t in T_list] for r in reactions_list ] 

In [9]:
to_chess('chess_data.txt', substances, reactions_dic, T_list, reactions_logKs, 'cemdata18 valid to 100C')

In [10]:
to_phreeqc('phreeqc_data.txt', engine, substances, reactions_list, reactions_dic, 'cemdata valid to 100C')